# Steel company problem in pyomo #

First, we need to install jupyter and pyomo, and import the pyomo package.

In [1]:
from pyomo.environ import *

Pyomo has the option of creating a ConcreteModel or an AbstractModel. We create a ConcreteModel here.

In [2]:
# create a model
model = ConcreteModel()

Definition of sets, parameters and decision variables. Note the difference between indexed and non-indexed parameters.

In [3]:
# Sets
model.PROD = Set(initialize=['BANDS', 'COILS'])  # products

# Parameters
model.T = Param(initialize=40)  # hours per week
model.c = Param(model.PROD, initialize={'BANDS': 25, 'COILS': 30}) # [$/t] profit per ton of product
model.B_max = Param(model.PROD, initialize={'BANDS': 6000, 'COILS': 4000})  # [t] max weekly production
model.t = Param(model.PROD, initialize={'BANDS': 200, 'COILS': 140})  # [t/h] Production rate

# Decision Variables
model.x = Var(model.PROD, domain=NonNegativeReals)  # [t] ton of products to produce

Constraints can be created in different ways.

In [4]:
# Constraints
def max_prod(model, i):
    return model.x[i] <= model.B_max[i]
model.max_prod = Constraint(model.PROD, rule=max_prod)

model.hours_week = Constraint(expr=sum(model.x[i]/model.t[i] for i in model.PROD) <= model.T)

Definition of the objective.

In [5]:
# Objective
def profit(model):
    return summation(model.c, model.x)
model.obj = Objective(rule=profit, sense=maximize)

Now we can print the model we have created.

In [6]:
model.pprint()

1 Set Declarations
    PROD : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {'BANDS', 'COILS'}

4 Param Declarations
    B_max : Size=2, Index=PROD, Domain=Any, Default=None, Mutable=False
        Key   : Value
        BANDS :  6000
        COILS :  4000
    T : Size=1, Index=None, Domain=Any, Default=None, Mutable=False
        Key  : Value
        None :    40
    c : Size=2, Index=PROD, Domain=Any, Default=None, Mutable=False
        Key   : Value
        BANDS :    25
        COILS :    30
    t : Size=2, Index=PROD, Domain=Any, Default=None, Mutable=False
        Key   : Value
        BANDS :   200
        COILS :   140

1 Var Declarations
    x : Size=2, Index=PROD
        Key   : Lower : Value : Upper : Fixed : Stale : Domain
        BANDS :     0 :  None :  None : False :  True : NonNegativeReals
        COILS :     0 :  None :  None : False :  True : NonNegativeReals

1 Objective Declarations
    obj

We solve the model in GLPK (using the solver GLPSOL) and we print the output variables.

In [7]:
SolverFactory('glpk', executable='/usr/local/bin/glpsol').solve(model).write()

# display solution
print('\nProfit = ', model.obj())

print('\nDecision Variables')
for i in model.x:
  print (str(model.x[i]), model.x[i].value)

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 192000.0
  Upper bound: 192000.0
  Number of objectives: 1
  Number of constraints: 4
  Number of variables: 3
  Number of nonzeros: 5
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.011626958847045898
# ----------------------------------------------------------
#   Solution Information
# ------------------------------